In [1]:
import numpy as np
import pandas as pd
import warnings
import geopy.distance
from sklearn.model_selection import train_test_split

warnings.filterwarnings('ignore')

from sklearn.metrics import mean_squared_error
import math

In [2]:
train_df = pd.read_csv("./data/train.csv")
test_df = pd.read_csv("./data/test.csv")

In [3]:
idx_trn = train_df.date.isin(train_df.date.unique()[:-7])
trn_df = train_df[idx_trn]
val_df = train_df[np.invert(idx_trn)]

# Function start

In [9]:
def data_prepare(df):
    
    try:
        y = df['18~20_ride']
    except:
        y = None
        
    df['date'] = pd.to_datetime(df['date'])
    df['weekday'] = df['date'].dt.weekday
    df = pd.get_dummies(df, columns=['weekday'])
    df['in_out'] = df['in_out'].map({'시내':0,'시외':1})


    coords_jejusi = (33.500770, 126.522761) #제주시의 위도 경도
    coords_seoquipo = (33.259429, 126.558217) #서귀포시의 위도 경도

    df['dis_jejusi'] = [geopy.distance.vincenty((df['latitude'].iloc[i], df['longitude'].iloc[i]), coords_jejusi).km for i in range(len(df))]
    df['dis_seoquipo'] = [geopy.distance.vincenty((df['latitude'].iloc[i], df['longitude'].iloc[i]), coords_seoquipo).km for i in range(len(df))]
    
    return df, y

In [7]:
trn, y_trn = data_prepare(trn_df)
val, y_val = data_prepare(val_df)

In [10]:
te, _ = data_prepare(test_df)

In [11]:
input_var=['in_out','latitude','longitude','6~7_ride', '7~8_ride', '8~9_ride', '9~10_ride',
       '10~11_ride', '11~12_ride', '6~7_takeoff', '7~8_takeoff', '8~9_takeoff',
       '9~10_takeoff', '10~11_takeoff', '11~12_takeoff','weekday_0', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4',
       'weekday_5', 'weekday_6', 'dis_jejusi', 'dis_seoquipo']
target=['18~20_ride']

In [12]:
x_trn = trn[input_var]
x_val = val[input_var]
x_te = te[input_var]

In [42]:
def esitimate_model(mdl, x_trn, y_trn, x_val, y_val):
    trns = mdl.predict(x_trn)
    vals = mdl.predict(x_val)

    print(math.sqrt(mean_squared_error(y_true=y_trn, y_pred=trns)))
    print(math.sqrt(mean_squared_error(y_true=y_val, y_pred=vals)))

In [40]:
from sklearn.ensemble import RandomForestRegressor
rf = ExtraTreesRegressor(max_depth=20, random_state=1991)

In [51]:
import lightgbm as lgb

In [68]:
m = lgb.LGBMRegressor(n_estimators=2000, num_leaves=31)

In [69]:
m.fit(x_trn, y_trn)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=2000, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [70]:
esitimate_model(m, x_trn, y_trn, x_val, y_val)

1.6630472310665352
2.734588194272103


# Sumbmission

In [72]:
test_df['18~20_ride'] = m.predict(x_te)
test_df[['id','18~20_ride']].to_csv("lgb_base.csv",index=False)